## FRISS with MFC

Implementation of the FRISS using the Media Frames Corpus (MFC) from Card et al. (2015). 

In [1]:
!pip install nltk

In [2]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [3]:
import os

os.listdir(os.getcwd())

['FRISS_srl.pkl',
 'training_metrics.csv',
 'README.md',
 'notebooks',
 'grid_search_metrics.csv',
 '.git',
 'assets',
 'test.csv',
 'friss',
 'models',
 '.ipynb_checkpoints',
 'data',
 '.gitignore',
 'frameaxis']

In [4]:
labels_path = "data/mfc/immigration_labeled.json"
codes_path = "data/mfc/codes.json"

In [5]:
# load data from path 
import json

with open(labels_path) as f:
    labels = json.load(f)

with open(codes_path) as f:
    codes = json.load(f)

In [6]:
import pandas as pd
from nltk.tokenize import sent_tokenize

# articles list
articles_list = []

# Iterate through the data to fill the DataFrame
for article_id, article_data in labels.items():
    annotations_data = article_data['annotations']

    irrelevant_dict = annotations_data['irrelevant']

    text = article_data['text']
    irrelevant = article_data['irrelevant']

    # if primary_frame is none set to 15.0
    if article_data['primary_frame'] is not None:
        primary_frame = str(article_data['primary_frame']).split(".")[0] + ".0"
    else:
        primary_frame = "15.0"

    # get primary frame from code
    primary_frame = str(codes[primary_frame])

    # split text into sentences using nltk library
    sentences = sent_tokenize(text)

    # iterate through sentences
    for sentence in sentences:
        article = {
            'article_id': article_id,
            'irrelevant': irrelevant,
            'text': sentence,
            'document_frame': primary_frame
        }

        articles_list.append(article)

# Create a DataFrame to store the results
df = pd.DataFrame(articles_list, columns=['article_id', 'irrelevant', 'text', 'document_frame'])


In [7]:
df

,article_id,irrelevant,text,document_frame
0,Immigration1.0-10005,0.0,IMM-10005\n\nPRIMARY\n\nImmigrants without HOP...,Quality of Life
1,Immigration1.0-10005,0.0,It mounted as students went around the room te...,Quality of Life
2,Immigration1.0-10005,0.0,Georgia Tech.,Quality of Life
3,Immigration1.0-10005,0.0,University of Georgia.,Quality of Life
4,Immigration1.0-10005,0.0,"""All I could say was, 'I'm planning to see if ...",Quality of Life
...,...,...,...,...
74463,Immigration1.0-9998,0.0,"Sue Brown, spokeswoman for the INS, said it's ...",Crime and Punishment
74464,Immigration1.0-9998,0.0,"""They love it,"" she said.",Crime and Punishment
74465,Immigration1.0-9998,0.0,"""They use these units to interview the people,...",Crime and Punishment
74466,Immigration1.0-9998,0.0,"""We do about 15 interviews a day,"" Brown said.",Crime and Punishment


In [8]:
df = df[df["irrelevant"] == False][["article_id", "text", "document_frame"]]

In [9]:
df.head()

,article_id,text,document_frame
0,Immigration1.0-10005,IMM-10005\n\nPRIMARY\n\nImmigrants without HOP...,Quality of Life
1,Immigration1.0-10005,It mounted as students went around the room te...,Quality of Life
2,Immigration1.0-10005,Georgia Tech.,Quality of Life
3,Immigration1.0-10005,University of Georgia.,Quality of Life
4,Immigration1.0-10005,"""All I could say was, 'I'm planning to see if ...",Quality of Life


In [10]:
# create for each code a col and fill with 1 if code is in code col
df = pd.concat([df, pd.get_dummies(df['document_frame'])], axis=1)

In [11]:
df.head()

,article_id,text,document_frame,Capacity and Resources,Crime and Punishment,Cultural Identity,Economic,External Regulation and Reputation,Fairness and Equality,Health and Safety,"Legality, Constitutionality, Jurisdiction",Morality,Other,Policy Prescription and Evaluation,Political,Public Sentiment,Quality of Life,Security and Defense
0,Immigration1.0-10005,IMM-10005\n\nPRIMARY\n\nImmigrants without HOP...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Immigration1.0-10005,It mounted as students went around the room te...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Immigration1.0-10005,Georgia Tech.,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Immigration1.0-10005,University of Georgia.,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Immigration1.0-10005,"""All I could say was, 'I'm planning to see if ...",Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [12]:
df.shape

(67480, 18)

### Create Dataset

In [13]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')

### Extract SRL Embeddings from articles

In [14]:
!pip install pycuda
!pip install allennlp allennlp-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 30.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2023.1-cp39-cp39-linux_x86_64.whl size=661891 sha256=69a923028332aad6d3854e0166b5d026014c5b4f0f6f7b1b4f7c7a11b168ea13
  Stored in directory: /root/.cache/pip/wheels/58/65/0d/8abe9d4a5fad9c5ba54c7e6b57bbc927a99517f3383a428f9d
Successfully built pycuda
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.5/298.5 kB 74.4 MB/s eta 0:00:00
  Prep

In [15]:
from allennlp.predictors.predictor import Predictor
import pandas as pd

In [16]:
# get name / id of cuda device
import pycuda.driver as cuda

cuda.init()
device = cuda.Device(0)
print(device.name())

NVIDIA RTX A6000


In [17]:
from tqdm.notebook import tqdm
import pandas as pd

def batched_extract_srl_components(batched_sentences, predictor):
    # Convert each sentence into the required format for the predictor
    batched_sentences = [{'sentence': sentence} for sentence in batched_sentences]

    # Prepare the batched input for the predictor
    batched_srl = predictor.predict_batch_json(batched_sentences)

    # Extract SRL components from the batched predictions
    results = []
    for index, srl in enumerate(batched_srl):
        sentence_results = []
        for verb_entry in srl['verbs']:
            arg_components = {'ARG0': [], 'ARG1': []}
            for i, tag in enumerate(verb_entry['tags']):
                if 'ARG0' in tag:
                    arg_components['ARG0'].append(srl['words'][i])
                elif 'ARG1' in tag:
                    arg_components['ARG1'].append(srl['words'][i])

            if arg_components['ARG0'] or arg_components['ARG1']:
                sentence_results.append({
                    'predicate': verb_entry['verb'],
                    'ARG0': ' '.join(arg_components['ARG0']),
                    'ARG1': ' '.join(arg_components['ARG1'])
                })

        if sentence_results:
            # add empty dict if predicate, arg0 or arg1 is empty
            if not sentence_results[0]['predicate']:
                results.append({'predicate': '', 'ARG0': '', 'ARG1': ''})
            elif not sentence_results[0]['ARG0']:
                results.append({'predicate': '', 'ARG0': '', 'ARG1': ''})
            elif not sentence_results[0]['ARG1']:
                results.append({'predicate': '', 'ARG0': '', 'ARG1': ''})
            else:
                results.append(sentence_results)    
        else:
            results.append([{'predicate': '', 'ARG0': '', 'ARG1': ''}])

    return results

def optimized_extract_srl(X, predictor, batch_size=32):
    all_results = []

    # Process sentences in batches
    for i in tqdm(range(0, len(X), batch_size), desc="Processing Batches"):
        batched_sentences = X[i:i+batch_size]

        batch_results = batched_extract_srl_components(batched_sentences, predictor)

        all_results.extend(batch_results)

    return pd.Series(all_results)

In [18]:
import pickle

def get_X_srl(X, recalculate=False, pickle_path="../notebooks/classifier/X_srl_filtered.pkl"):
    """
    Returns the X_srl either by loading from a pickled file or recalculating.
    """
    if recalculate or not os.path.exists(pickle_path):
        print("Recalculate SRL")
        # Load predictor
        predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz", cuda_device=0)

        # make sentences max 480 chars long
        X = X.apply(lambda x: x[:480])

        X_srl = optimized_extract_srl(X, predictor, batch_size=32)
        with open(pickle_path, 'wb') as f:
            pickle.dump(X_srl, f)
    else:
        print("Load SRL from Pickle")
        with tqdm(total=os.path.getsize(pickle_path)) as pbar:
            with open(pickle_path, 'rb') as f:
                X_srl = pickle.load(f)
                pbar.update(os.path.getsize(pickle_path))
                
    return X_srl

In [19]:
# get_X_srl(df["text"], recalculate=False, pickle_path="../notebooks/FRISS_srl.pkl")

# GPU

In [20]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

def free_gpu():
    print(torch.cuda.mem_get_info())
    print(torch.cuda.memory_summary())

Using device: cuda


In [21]:
import torch
import gc

def list_gpu_tensors():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if obj.is_cuda:
                    obj = obj.cpu()
                    obj = obj.to("cpu")
                    print(type(obj), obj.size())
        except:
            pass

        
list_gpu_tensors()

/usr/local/lib/python3.9/dist-packages/torch/distributed/distributed_c10d.py:181: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


# Dataset

In [89]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd

class ArticleDataset(Dataset):
    def __init__(self, X, X_srl, tokenizer, labels=None, max_sentences_per_article=32, max_sentence_length=32, max_args_per_sentence=10, max_arg_length=16):
        self.X = X  # DataFrame where each row has multiple sentences
        self.X_srl = X_srl  # DataFrame where each row has multiple dictionaries for SRL
        self.labels = labels  # DataFrame where each row has a list of lists of integers
        
        self.tokenizer = tokenizer
        self.max_sentences_per_article = max_sentences_per_article
        self.max_sentence_length = max_sentence_length
        self.max_args_per_sentence = max_args_per_sentence
        self.max_arg_length = max_arg_length

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sentences = self.X.iloc[idx]
        srl_data = self.X_srl.iloc[idx]
        labels = self.labels.iloc[idx]

        # Tokenize sentences
        sentence_ids = [self.tokenizer.encode(sentence, add_special_tokens=True, max_length=self.max_sentence_length, truncation=True, padding='max_length') for sentence in sentences]
        
        sentence_ids += [[0] * self.max_sentence_length] * (self.max_sentences_per_article - len(sentence_ids))
        sentence_ids = sentence_ids[:self.max_sentences_per_article]
        
        # Process SRL data
        predicates, arg0s, arg1s = [], [], []
        for srl_items in srl_data:

            sentence_predicates, sentence_arg0s, sentence_arg1s = [], [], []

            # if srl_items is not list pack it into a list
            if not isinstance(srl_items, list):
                srl_items = [srl_items]

            for item in srl_items:
                p = self.tokenizer.encode(item["predicate"], add_special_tokens=True, max_length=self.max_arg_length, truncation=True, padding='max_length')
                a0 = self.tokenizer.encode(item["ARG0"], add_special_tokens=True, max_length=self.max_arg_length, truncation=True, padding='max_length')
                a1 = self.tokenizer.encode(item["ARG1"], add_special_tokens=True, max_length=self.max_arg_length, truncation=True, padding='max_length')
                
                sentence_predicates.append(p)
                sentence_arg0s.append(a0)
                sentence_arg1s.append(a1)

            # Padding
            for _ in range(self.max_args_per_sentence - len(srl_items)):
                sentence_predicates.append([0] * self.max_arg_length)
                sentence_arg0s.append([0] * self.max_arg_length)
                sentence_arg1s.append([0] * self.max_arg_length)

            # pad to max_args_per_sentence
            sentence_predicates = sentence_predicates[:self.max_args_per_sentence]
            sentence_arg0s = sentence_arg0s[:self.max_args_per_sentence]
            sentence_arg1s = sentence_arg1s[:self.max_args_per_sentence]

            predicates.append(sentence_predicates)
            arg0s.append(sentence_arg0s)
            arg1s.append(sentence_arg1s)

        # Truncate or pad SRL items to max_sentences_per_article
        srl_padding = [[0] * self.max_arg_length] * self.max_args_per_sentence
        predicates = (predicates + [srl_padding] * self.max_sentences_per_article)[:self.max_sentences_per_article]
        arg0s = (arg0s + [srl_padding] * self.max_sentences_per_article)[:self.max_sentences_per_article]
        arg1s = (arg1s + [srl_padding] * self.max_sentences_per_article)[:self.max_sentences_per_article]

        data = {
            'sentence_ids': torch.tensor(sentence_ids, dtype=torch.long),
            'predicate_ids': torch.tensor(predicates, dtype=torch.long),
            'arg0_ids': torch.tensor(arg0s, dtype=torch.long),
            'arg1_ids': torch.tensor(arg1s, dtype=torch.long),
            'labels': torch.tensor(labels[0], dtype=torch.long)
        }

        return data


In [90]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split



In [91]:
def custom_collate_fn(batch):
    # Extract individual lists from the batch
    sentence_ids = [item['sentence_ids'] for item in batch]
    predicate_ids = [item['predicate_ids'] for item in batch]
    arg0_ids = [item['arg0_ids'] for item in batch]
    arg1_ids = [item['arg1_ids'] for item in batch]
    labels = [item['labels'] for item in batch]
    
    # Pad each list
    sentence_ids = torch.nn.utils.rnn.pad_sequence(sentence_ids, batch_first=True, padding_value=0)
    predicate_ids = torch.nn.utils.rnn.pad_sequence(predicate_ids, batch_first=True, padding_value=0)
    arg0_ids = torch.nn.utils.rnn.pad_sequence(arg0_ids, batch_first=True, padding_value=0)
    arg1_ids = torch.nn.utils.rnn.pad_sequence(arg1_ids, batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0)

    # Conditionally extract and add labels
    output_dict = {
        'sentence_ids': sentence_ids,
        'predicate_ids': predicate_ids,
        'arg0_ids': arg0_ids,
        'arg1_ids': arg1_ids,
        'labels': labels
    }

    return output_dict

def preprocess_df(df, recalculate_srl=False, pickle_path="../notebooks/FRISS_srl.pkl"):
    # reset index of df
    df = df.reset_index(drop=True)

    # Get X_srl
    X_srl = get_X_srl(df["text"], recalculate=recalculate_srl, pickle_path=pickle_path)

    # reset index of X_srl
    X_srl = X_srl.reset_index(drop=True)

    # Columns to be one-hot encoded in y_subset
    y_cols = ['Capacity and Resources', 'Crime and Punishment', 'Cultural Identity', 
            'Economic', 'External Regulation and Reputation', 'Fairness and Equality', 
            'Health and Safety', 'Legality, Constitutionality, Jurisdiction', 
            'Morality', 'Other', 'Policy Prescription and Evaluation', 'Political', 
            'Public Sentiment', 'Quality of Life', 'Security and Defense']

    # Creating y_subset
    y_subset = df.groupby('article_id')[y_cols].apply(lambda x: x.values.tolist()).reset_index(name='encoded_values')
    y_subset = y_subset['encoded_values']

    # Aggregating 'text' column in df into a list of strings for each article_id
    X_subset = df.groupby('article_id')['text'].apply(list).reset_index(name='text')
    X_subset = X_subset['text']

    # Assuming X_srl follows the same index order as df
    X_srl_subset = X_srl.groupby(df['article_id']).apply(lambda x: x.values.tolist()).reset_index(name='srl_values')
    X_srl_subset = X_srl_subset['srl_values']

    return X_subset, X_srl_subset, y_subset

def get_datasets_dataloaders(df, tokenizer, recalculate_srl=False, pickle_path="../notebooks/FRISS_srl.pkl", batch_size=16, max_sentences_per_article=32, max_sentence_length=32, max_arg_length=16):
    
    X_subset, X_srl_subset, y_subset = preprocess_df(df, recalculate_srl=recalculate_srl, pickle_path=pickle_path)

    # Len
    print("X:", len(X_subset))
    print("X_srl:", len(X_srl_subset))
    print("y:", len(y_subset))

    print("CREATING DATASETS")
    test_size = 0.1
    
    # Assuming X, X_srl, and y are already defined and have the same number of samples
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=test_size, random_state=42)
    
    print("TRAIN TEST SPLIT DONE")
    
    X_srl_train, X_srl_test, _, _ = train_test_split(X_srl_subset, y_subset, test_size=test_size, random_state=42)

    # Create the dataset
    train_dataset = ArticleDataset(X_train, X_srl_train, tokenizer, y_train, max_sentences_per_article, max_sentence_length, max_arg_length)
    test_dataset = ArticleDataset(X_test, X_srl_test, tokenizer, y_test, max_sentences_per_article, max_sentence_length, max_arg_length)

    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn, drop_last=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn, drop_last=True)
    
    print("CREATION DONE")
    return train_dataset, test_dataset , train_dataloader, test_dataloader

In [93]:
def get_article_dataloader(article, tokenizer, batch_size=1):
    X = pd.Series([article])
    y = None  # No labels for this single article
    
    predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz", cuda_device=0)
    # Directly use the optimized_extract_srl function since we don't need to cache for single articles
    X_srl = optimized_extract_srl(X, predictor)
    
    # Create the dataset
    dataset = ArticleDataset(X, X_srl, tokenizer, y)
    
    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
    
    return dataloader

In [94]:
def get_test_dataloader(X, tokenizer, batch_size=4):
    y = None
    
    predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz", cuda_device=0)
    # Directly use the optimized_extract_srl function since we don't need to cache for single articles
    X_srl = optimized_extract_srl(X, predictor)
    
    # Create the dataset
    dataset = ArticleDataset(X, X_srl, tokenizer, y)
    
    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
    
    return dataloader

# PyTorch Model
The Model consist out of various Layers.

1. SRL_Embedding
2. Autoencoder
3. FRISSLoss
4. Unsupervised
5. Supervised
6. FRISS

In [95]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

## 1. SRL_Embeddings

The layer takes tensors of token IDs with the shape [batch_size, max_num_sentences, max_num_tokens] for the sentence, predicates, arg0 and arg1 and returns for each sentence an embedding with shape [batch_size, embedding_dim] for the sentence, predicate, arg0 and arg1. 

The single embedding for the sentence is extracted by taking the [CLS] token embedding. For the predicate, arg0 and arg1 by taking the mean over all word embeddings in this list of tokens. 

> Possible improvements: Better way of extracting the single embedding for predicate, arg0 and arg1.

In [96]:
from transformers import BertModel
import torch.nn as nn
import torch

class SRL_Embeddings(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased"):
        super(SRL_Embeddings, self).__init__()
        self.bert_model = BertModel.from_pretrained(bert_model_name)
        self.embedding_dim = 768  # for bert-base-uncased

    def forward(self, sentence_ids, predicate_ids, arg0_ids, arg1_ids):
        with torch.no_grad():
            # Sentence embeddings
            sentence_embeddings = self.bert_model(sentence_ids.view(-1, sentence_ids.size(-1)))[0]
            sentence_embeddings = sentence_embeddings.view(sentence_ids.size(0), sentence_ids.size(1), -1, self.embedding_dim)
            sentence_embeddings = sentence_embeddings.mean(dim=2)

            # Predicate embeddings
            predicate_embeddings = self.bert_model(predicate_ids.view(-1, predicate_ids.size(-1)))[0]
            predicate_embeddings = predicate_embeddings.view(predicate_ids.size(0), predicate_ids.size(1), predicate_ids.size(2), -1, self.embedding_dim)
            predicate_embeddings = predicate_embeddings.mean(dim=3)

            # ARG0 embeddings
            arg0_embeddings = self.bert_model(arg0_ids.view(-1, arg0_ids.size(-1)))[0]
            arg0_embeddings = arg0_embeddings.view(arg0_ids.size(0), arg0_ids.size(1), arg0_ids.size(2), -1, self.embedding_dim)
            arg0_embeddings = arg0_embeddings.mean(dim=3)

            # ARG1 embeddings
            arg1_embeddings = self.bert_model(arg1_ids.view(-1, arg1_ids.size(-1)))[0]
            arg1_embeddings = arg1_embeddings.view(arg1_ids.size(0), arg1_ids.size(1), arg1_ids.size(2), -1, self.embedding_dim)
            arg1_embeddings = arg1_embeddings.mean(dim=3)

        return sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings
    
# Generate dummy data for the SRL_Embeddings
batch_size = 2
num_sentences = 12
sentence_length = 8
num_args = 9
predicate_length = 8
arg0_length = 8
arg1_length = 8

# Dummy data for sentences, predicates, arg0, and arg1
sentence_ids = torch.randint(0, 10000, (batch_size, num_sentences, sentence_length))
predicate_ids = torch.randint(0, 10000, (batch_size, num_sentences, num_args, predicate_length))
arg0_ids = torch.randint(0, 10000, (batch_size, num_sentences, num_args, arg0_length))
arg1_ids = torch.randint(0, 10000, (batch_size, num_sentences, num_args, arg1_length))

srl_embeddings = SRL_Embeddings()

sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings = srl_embeddings(sentence_ids, predicate_ids, arg0_ids, arg1_ids)

print("Inputs shapes: ", sentence_ids.shape, predicate_ids.shape, arg0_ids.shape, arg1_ids.shape)
print("Outputs shapes: ", sentence_embeddings.shape, predicate_embeddings.shape, arg0_embeddings.shape, arg1_embeddings.shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Inputs shapes:  torch.Size([2, 12, 8]) torch.Size([2, 12, 9, 8]) torch.Size([2, 12, 9, 8]) torch.Size([2, 12, 9, 8])
Outputs shapes:  torch.Size([2, 12, 768]) torch.Size([2, 12, 9, 768]) torch.Size([2, 12, 9, 768]) torch.Size([2, 12, 9, 768])


## 2. Autoencoder

The layer takes tensors for `v` (size: [batch_size, embedding_dim]), `v_sentence` (size: [batch_size, embedding_dim]), `tau` (type: _float_), and `identifier` (type: _str_). Where `v` is the embedding of either predicate, arg0 or arg1 identified by the `identifier` parameter. The `v_sentence` is the sentence embedding and `tau` defined the tau for annealing the gumpel softmax.

The forward function returns `vhat` (size: [batch_size, embedding_dim]), `dz` (size: [batch_size, embedding_dim]), `gz` (size: [batch_size, embedding_dim]) and `F` (size: [K, embedding_dim]).

- `vhat`: Reconstructed embedding of SRL
- `dz`: Descriptor weights
- `gz`: Gumbel softmax from logits
- `F`: Dictionary

In [97]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CombinedAutoencoder(nn.Module):
    def __init__(self, D_w, D_h, K, dropout_prob=0.3):
        super(CombinedAutoencoder, self).__init__()
        
        self.D_h = D_h
        self.K = K
        
        # Shared feed-forward layer for all views
        self.feed_forward_shared = nn.Linear(2 * D_w, D_h)
        
        # Unique feed-forward layers for each view
        self.feed_forward_unique = nn.ModuleDict({
            'a0': nn.Linear(D_h, K),
            'p': nn.Linear(D_h, K),
            'a1': nn.Linear(D_h, K),
        })

        # Initializing F matrices for each view
        self.F_matrices = nn.ParameterDict({
            'a0': nn.Parameter(torch.randn(K, D_w)),
            'p': nn.Parameter(torch.randn(K, D_w)),
            'a1': nn.Parameter(torch.randn(K, D_w)),
        })

        # Additional layers and parameters
        self.dropout = nn.Dropout(dropout_prob)
        self.batch_norm = nn.BatchNorm1d(D_h)
        self.activation = nn.ReLU()
        self.activation2 = nn.Sigmoid()
    
    # try softmax
    def gumbel_softmax(self, logits, tau: float = 1, hard: bool = False, eps: float = 1e-10, threshold = 0.5, dim: int = 1):
        gumbels = (
            -torch.empty_like(logits, memory_format=torch.legacy_contiguous_format).exponential_().log()
        )  # ~Gumbel(0,1)

        #gumbels = (torch.log(logits) + (gumbels/tau))  # ~Gumbel(logits,tau)
        gumbels = (logits + gumbels) / tau
        y_soft = gumbels.softmax(dim)

        if hard:
            indices = (y_soft > threshold).nonzero(as_tuple=True)
            y_hard = torch.zeros_like(logits, memory_format=torch.legacy_contiguous_format)
            y_hard[indices[0], indices[1]] = 1.0
            ret = y_hard - y_soft.detach() + y_soft
        else:
            ret = y_soft

        return ret

    def forward(self, v_p, v_a0, v_a1, v_sentence, tau):
        h_p, h_a0, h_a1 = self.process_through_shared(v_p, v_a0, v_a1, v_sentence)

        logits_p = self.feed_forward_unique['p'](h_p)
        logits_a0 = self.feed_forward_unique['a0'](h_a0)
        logits_a1 = self.feed_forward_unique['a1'](h_a1)
        
        dz_p = torch.softmax(logits_p, dim=1)
        dz_a0 = torch.softmax(logits_a0, dim=1)
        dz_a1 = torch.softmax(logits_a1, dim=1)
        
        gz_p = self.gumbel_softmax(dz_p, tau=tau, hard=False)
        gz_a0 = self.gumbel_softmax(dz_a0, tau=tau, hard=False)
        gz_a1 = self.gumbel_softmax(dz_a1, tau=tau, hard=False)

        vhat_p = torch.matmul(gz_p, self.F_matrices['p'])
        vhat_a0 = torch.matmul(gz_a0, self.F_matrices['a0'])
        vhat_a1 = torch.matmul(gz_a1, self.F_matrices['a1'])

        return {
            "p": {"vhat": vhat_p, "d": dz_p, "g": gz_p, "F": self.F_matrices['p']},
            "a0": {"vhat": vhat_a0, "d": dz_a0, "g": gz_a0, "F": self.F_matrices['a0']},
            "a1": {"vhat": vhat_a1, "d": dz_a1, "g": gz_a1, "F": self.F_matrices['a1']}
        }
        
    def process_through_shared(self, v_p, v_a0, v_a1, v_sentence):
        concatenated_p = torch.cat((v_p, v_sentence), dim=-1)
        concatenated_a0 = torch.cat((v_a0, v_sentence), dim=-1)
        concatenated_a1 = torch.cat((v_a1, v_sentence), dim=-1)
        
        # Concatenate them along the batch dimension for a single pass through the shared layer
        stacked_embeddings = torch.cat([concatenated_p, concatenated_a0, concatenated_a1], dim=0)
        
        #h_shared = self.dropout(stacked_embeddings)
        h_shared = self.feed_forward_shared(stacked_embeddings)
        
        # Splitting them back to individual embeddings
        batch_size = v_p.shape[0]
        h_shared = h_shared.view(3, batch_size, self.D_h)
        
        h_p, h_a0, h_a1 = h_shared[0], h_shared[1], h_shared[2]
        return h_p, h_a0, h_a1

# Mock Data Preparation
D_h = 768
batch_size = 2
embedding_dim = 768
K = 20
tau = 0.9

# Generating mock embeddings for article, predicate, ARG0, ARG1, and their corresponding sentence embeddings
article_embedding = torch.randn(batch_size, embedding_dim)
v_p = torch.randn(batch_size, embedding_dim)
v_a0 = torch.randn(batch_size, embedding_dim)
v_a1 = torch.randn(batch_size, embedding_dim)

# Testing CombinedAutoencoder
autoencoder = CombinedAutoencoder(embedding_dim, D_h, K)
outputs = autoencoder(v_p, v_a0, v_a1, article_embedding, tau)

# Check shapes of the outputs
print("Output shapes:")
for key, value in outputs.items():
    print(f"{key} -> vhat: {value['vhat'].shape}, d: {value['d'].shape}, g: {value['g'].shape}, F: {value['F'].shape}")

# check if tensor have nan values
def check_nan(tensor):
    # if tensor has any nan values, return True
    if torch.isnan(tensor).any():
        return True
    else:
        return False

# Check if any of the outputs have NaN values
print("NaN values:")
for key, value in outputs.items():
    print(f"{key} -> vhat: {check_nan(value['vhat'])}, d: {check_nan(value['d'])}, g: {check_nan(value['g'])}, F: {check_nan(value['F'])}")

Output shapes:
p -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])
a0 -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])
a1 -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])
NaN values:
p -> vhat: False, d: False, g: False, F: False
a0 -> vhat: False, d: False, g: False, F: False
a1 -> vhat: False, d: False, g: False, F: False


## 3. FRISSLoss

The layer calculates the unsupervised loss for predicate, arg0 and arg1. 

The forward function takes as input 3 dicts with the parameters `v`, `v_hat`, `g` and `F`. Where `v` is the embedding of the predicate, arg0 or arg1. The `v_hat` (size: [batch_size, embedding_dim]) is the reconstructed embedding for the predicate, arg0 and arg1. The `g` is the gumbel softmax result (size: [batch_size, embedding_dim]). The `F` (size: [K, embedding_dim]) which is the descriptor dictionary.

The layer returns the loss for each batch. So the output is [batch_size].

In [98]:
class FRISSLoss(nn.Module):
    def __init__(self, lambda_orthogonality, M, t):
        super(FRISSLoss, self).__init__()
        
        self.lambda_orthogonality = lambda_orthogonality
        self.M = M
        self.t = t
        self.triplet_loss = nn.TripletMarginLoss(margin=M)

    def contrastive_loss(self, v, vhat, negatives):
        batch_size = vhat.size(0)
        N = negatives.size(0)
        loss = torch.zeros(batch_size, device=v.device)

        # Calculate true distance between reconstructed and real embeddings
        true_distance = self.l2(vhat, v)

        for i in range(N):  # loop over each element in "negatives"
            
            # Tranform negative from [embedding dim] to [batch size, embedding_dim] 
            negative = negatives[i, :].expand(v.size(0), -1)

            # Calculate negative distance for current negative embedding
            negative_distance = self.l2(vhat, negative)

            # Compute loss based on the provided logic: l2(vhat, v) + 1 + l2(vhat, negative) and clamp to 0 if below 0
            current_loss = 1 + true_distance - negative_distance
            loss += torch.clamp(current_loss, min=0.0)

        # Normalize the total loss by N
        return loss / N

    
    def l2(self, u, v):
        return torch.sqrt(torch.sum((u - v) ** 2, dim=1))
    
    def focal_triplet_loss_WRONG(self, v, vhat_z, g, F):
        losses = []
        for i in range(F.size(0)):  # Iterate over each negative example
            # For each negative, compute the loss against the anchor and positive
            loss = self.triplet_loss(vhat_z, v, F[i].unsqueeze(0).expand(v.size(0), -1))
            losses.append(loss)

        loss_tensor = torch.stack(losses) 
        loss = loss_tensor.mean(dim=0).mean()
        return loss
    
    def focal_triplet_loss(self, v, vhat_z, g, F):
        _, indices = torch.topk(g, self.t, largest=False, dim=1)

        F_t = torch.stack([F[indices[i]] for i in range(g.size(0))])
        
        g_tz = torch.stack([g[i, indices[i]] for i in range(g.size(0))])
                    
        g_t = g_tz / g_tz.sum(dim=1, keepdim=True)
        
        # if division by zero set all nan values to 0
        g_t[torch.isnan(g_t)] = 0
        
        m_t = self.M * ((1 - g_t)**2)

        # Initializing loss
        loss = torch.zeros_like(v[:, 0])
        
        # Iteratively adding to the loss for each negative embedding
        for i in range(self.t):
            current_v_t = F_t[:, i]
            current_m_t = m_t[:, i]
            
            current_loss = current_m_t + self.l2(vhat_z, v) - self.l2(vhat_z, current_v_t)
            
            loss += torch.max(torch.zeros_like(current_loss), current_loss)
             
        # Normalizing
        loss = loss / self.t
        return loss

    def orthogonality_term(self, F, reg=1e-4):
        gram_matrix = torch.mm(F, F.T)  # Compute the Gram matrix F * F^T
        identity_matrix = torch.eye(gram_matrix.size(0), device=gram_matrix.device)  # Create an identity matrix
        ortho_loss = (gram_matrix - identity_matrix).abs().sum()
        return ortho_loss


    def forward(self, p, a0, a1, p_negatives, a0_negatives, a1_negatives):
        # Extract components from dictionary for predicate p
        v_p, vhat_p, d_p, g_p, F_p = p["v"], p["vhat"], p["d"], p["g"], p["F"]
        
        # Extract components from dictionary for ARG0
        v_a0, vhat_a0, d_a0, g_a0, F_a0 = a0["v"], a0["vhat"], a0["d"], a0["g"], a0["F"]

        # Extract components from dictionary for ARG1
        v_a1, vhat_a1, d_a1, g_a1, F_a1 = a1["v"], a1["vhat"], a1["d"], a1["g"], a1["F"]
        
         # Calculate losses for predicate
        Ju_p = self.contrastive_loss(v_p, vhat_p, p_negatives)        
        Jt_p = self.focal_triplet_loss(v_p, vhat_p, g_p, F_p)
        
        Jz_p = Ju_p + Jt_p + self.lambda_orthogonality * self.orthogonality_term(F_p) ** 2
        #print(Ju_p, Jt_p, self.orthogonality_term(F_p))
        # Calculate losses for ARG0
        Ju_a0 = self.contrastive_loss(v_a0, vhat_a0, a0_negatives)
        Jt_a0 = self.focal_triplet_loss(v_a0, vhat_a0, g_a0, F_a0)
        Jz_a0 = Ju_a0 + Jt_a0 + self.lambda_orthogonality * self.orthogonality_term(F_a0) ** 2
        
        # Calculate losses for ARG1
        Ju_a1 = self.contrastive_loss(v_a1, vhat_a1, a1_negatives)
        Jt_a1 = self.focal_triplet_loss(v_a1, vhat_a1, g_a1, F_a1)
        Jz_a1 = Ju_a1 + Jt_a1 + self.lambda_orthogonality * self.orthogonality_term(F_a1) ** 2
        
        if torch.isnan(Jz_p).any():
            print("Jz_p has nan")
            
        if torch.isnan(Jz_a0).any():
            print("Jz_a0 has nan")
            
        if torch.isnan(Jz_a1).any():
            print("Jz_a1 has nan")
        
        # Aggregate the losses
        loss = Jz_p + Jz_a0 + Jz_a1
        
        return loss


# Mock Data Preparation
batch_size = 2
embedding_dim = 768
K = 15  # Number of frames/descriptors

# Generating mock embeddings for article, predicate, ARG0, ARG1 and their reconstructions
article_embedding = torch.randn(batch_size, embedding_dim)
v_p = torch.randn(batch_size, embedding_dim)
vhat_p = torch.randn(batch_size, embedding_dim)

v_a0 = torch.randn(batch_size, embedding_dim)
vhat_a0 = torch.randn(batch_size, embedding_dim)

v_a1 = torch.randn(batch_size, embedding_dim)
vhat_a1 = torch.randn(batch_size, embedding_dim)

# Generating mock descriptor weights and descriptor matrices for predicate, ARG0, ARG1
d_p = torch.randn(batch_size, K)
d_a0 = torch.randn(batch_size, K)
d_a1 = torch.randn(batch_size, K)

F_p = torch.randn(K, embedding_dim)
F_a0 = torch.randn(K, embedding_dim)
F_a1 = torch.randn(K, embedding_dim)

g_p = torch.randn(batch_size, K)
g_a0 = torch.randn(batch_size, K)
g_a1 = torch.randn(batch_size, K)

# Generating some negative samples (let's assume 5 negative samples per batch entry)
num_negatives = 8
negatives_p = torch.randn(num_negatives, embedding_dim)
negatives_a0 = torch.randn(num_negatives, embedding_dim)
negatives_a1 = torch.randn(num_negatives, embedding_dim)

# Initialize loss function
lambda_orthogonality = 1e-3

t = 8  # Number of descriptors with smallest weights for negative samples
M = t

loss_fn = FRISSLoss(lambda_orthogonality, M, t)

# Organizing inputs into dictionaries
p = {"v": v_p, "vhat": vhat_p, "d": d_p, "g": g_p, "F": F_p}
a0 = {"v": v_a0, "vhat": vhat_a0, "d": d_a0, "g": g_a0, "F": F_a0}
a1 = {"v": v_a1, "vhat": vhat_a1, "d": d_a1, "g": g_a1, "F": F_a1}

loss_fn = FRISSLoss(lambda_orthogonality, M, t)
loss = loss_fn(p, a0, a1, negatives_p, negatives_a0, negatives_a1)
print("FRiSSLoss output:", loss)

FRiSSLoss output: tensor([822901.6250, 822901.3750])


## 4. FRISSUnsupervised

The `FRISSUnsupervised` layer integrates multiple autoencoders and the previously described `FRISSLoss` layer to achieve an unsupervised learning process over the predicates and their arguments.

### Forward Method:

**Inputs**:
1. **v_p**: Embedding of the predicate with size: [batch_size, D_w].
2. **v_a0**: Embedding of the ARG0 (first argument) with size: [batch_size, D_w].
3. **v_a1**: Embedding of the ARG1 (second argument) with size: [batch_size, D_w].
4. **v_article**: Embedding of the article with size: [batch_size, D_w].
5. **negatives**: Tensor containing negative samples with size: [batch_size, num_negatives, D_w].
6. **tau**: A scalar parameter for the Gumbel softmax in the autoencoder.

**Outputs**:
- A dictionary `results` containing:
    - **loss**: A tensor representing the combined unsupervised loss over the batch with size: [batch_size].
    - **p**: Dictionary containing components for the predicate, including reconstructed embedding (`vhat`), descriptor weights (`d`), Gumbel softmax result (`g`), and the descriptor matrix (`F`).
    - **a0**: Same as `p` but for ARG0.
    - **a1**: Same as `p` but for ARG1.

In [99]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Assuming you have already defined CombinedAutoencoder and its methods as provided earlier.

class FRISSUnsupervised(nn.Module):
    def __init__(self, D_w, D_h, K, num_frames, lambda_orthogonality, M, t, dropout_prob=0.3):
        super(FRISSUnsupervised, self).__init__()
        
        self.loss_fn = FRISSLoss(lambda_orthogonality, M, t)      
        
        # Using the CombinedAutoencoder instead of individual Autoencoders
        self.combined_autoencoder = CombinedAutoencoder(D_w, D_h, K, dropout_prob=dropout_prob)

    def forward(self, v_p, v_a0, v_a1, v_sentence, p_negatives, a0_negatives, a1_negatives, tau):
        outputs = self.combined_autoencoder(v_p, v_a0, v_a1, v_sentence, tau)

        outputs_p = outputs["p"]
        outputs_p["v"] = v_p
        
        outputs_a0 = outputs["a0"]
        outputs_a0["v"] = v_a0
        
        outputs_a1 = outputs["a1"]
        outputs_a1["v"] = v_a1
        
        loss = self.loss_fn(
            outputs_p,
            outputs_a0, 
            outputs_a1, 
            p_negatives, a0_negatives, a1_negatives
        )

        results = {
            "loss": loss,
            "p": outputs["p"],
            "a0": outputs["a0"],
            "a1": outputs["a1"]
        }
        
        return results

# Mock Data Preparation
D_h = 768
batch_size = 2
embedding_dim = 768
K = 20
num_frames = 15
tau = 0.9
lambda_orthogonality = 0.1  # Placeholder value, please replace with your actual value
M = 7  # Placeholder value, please replace with your actual value
t = 7  # Placeholder value, please replace with your actual value

# Generating mock embeddings for article, predicate, ARG0, ARG1, and their corresponding sentence embeddings
article_embedding = torch.randn(batch_size, embedding_dim)
v_p = torch.randn(batch_size, embedding_dim)
v_a0 = torch.randn(batch_size, embedding_dim)
v_a1 = torch.randn(batch_size, embedding_dim)

# Generating some negative samples (let's assume 5 negative samples per batch entry)
num_negatives = 10
negatives_p = torch.randn(num_negatives, embedding_dim)
negatives_a0 = torch.randn(num_negatives, embedding_dim)
negatives_a1 = torch.randn(num_negatives, embedding_dim)

# Testing FRISSUnsupervised
unsupervised_module = FRISSUnsupervised(embedding_dim, D_h, K, num_frames, lambda_orthogonality, M, t)
results = unsupervised_module(v_p, v_a0, v_a1, article_embedding, negatives_p, negatives_a0, negatives_a1, tau)

# Print the results' shapes for verification
print("Results' Shapes:")
for key, value in results.items():
    if key == "loss":
        print(f"{key}: {value}")
    else:
        print(f"{key} -> vhat: {value['vhat'].shape}, d: {value['d'].shape}, g: {value['g'].shape}, F: {value['F'].shape}")


Results' Shapes:
loss: tensor([1.7158e+08, 1.7158e+08], grad_fn=<AddBackward0>)
p -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])
a0 -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])
a1 -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])


## 5. FRISSSupervised

The layer takes the embeddings from the args and the sentence and predicts frames. 

The embeddings for the args are averaged for each arg individually and then averaged on args level. The final embedding is feed into a linear layer and passed through a sigmoid function. 

The sentence embedding is feed into a linear layer and then into a relu function. After again in a linear function and then averaged. The average embeddung is again feed into a linear layer and lastly in a signoid function. 

It returns a span and sentence based prediction of shape [batch_size, num_frames].

In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FRISSSupervised(nn.Module):
    def __init__(self, D_w, K, num_frames, dropout_prob=0.3):
        super(FRISSSupervised, self).__init__()

        self.D_w = D_w
                
        self.softmax = nn.Softmax(dim=1)

        self.feed_forward_sentence1 = nn.Linear(D_w, D_w)
        self.feed_forward_sentence2 = nn.Linear(D_w, num_frames)

        self.relu = nn.ReLU()
        
    def forward(self, d_p, d_a0, d_a1, vs):
        # Span-based Classification   

        # aggregate the SRL descriptors to have one descriptor per sentence
        d_p = d_p.mean(dim=2)
        d_a0 = d_a0.mean(dim=2)
        d_a1 = d_a1.mean(dim=2)

        # take the mean over descriptors
        d_v = (d_p + d_a0 + d_a1) / 3

        # feed in softmax
        # yu_hat = self.softmax(d_v) # do not use as we use crossentropyloss

        # Sentence-based Classification

        ws = self.relu(self.feed_forward_sentence1(vs))

        ws = self.feed_forward_sentence2(ws)

        # mean over sentences
        ws = ws.mean(dim=1)

        # softmax
        # ys_hat = self.softmax(ws) # do not use as we use crossentropyloss

        return d_v, ws


# Mock Data Preparation

batch_size = 2
embedding_dim = 768
num_frames = 15  # Assuming the number of frames is equal to K for simplicity
num_sentences = 32
K = 15
num_args = 9

# Generating mock dsz representations for predicate, ARG0, ARG1
d_p = torch.randn(batch_size, num_sentences, num_args, K)
d_a0 = torch.randn(batch_size, num_sentences, num_args, K)
d_a1 = torch.randn(batch_size, num_sentences, num_args, K) 

# Adjusting the num_heads parameter
srl_heads = 4
sentence_heads = 8

# Adjust the mock sentence embeddings shape
vs = torch.randn(batch_size, num_sentences, embedding_dim)

# Initialize and test the supervised module
supervised_module = FRISSSupervised(embedding_dim, K, num_frames)

# Forward pass the mock data
yu_hat, ys_hat = supervised_module(d_p, d_a0, d_a1, vs)
yu_hat.shape, ys_hat.shape

(torch.Size([2, 32, 15]), torch.Size([2, 15]))

## 6. FRISS

In [101]:
import torch.nn as nn

class FRISS(nn.Module):
    def __init__(self, embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob=0.3, bert_model_name="bert-base-uncased"):
        super(FRISS, self).__init__()
        
        # Aggregation layer replaced with SRL_Embeddings
        self.aggregation = SRL_Embeddings(bert_model_name)
        
        # Unsupervised training module
        self.unsupervised = FRISSUnsupervised(embedding_dim, D_h, K, num_frames, lambda_orthogonality, M, t, dropout_prob=dropout_prob)
        
        # Supervised training module
        self.supervised = FRISSSupervised(embedding_dim, K, num_frames, dropout_prob=dropout_prob)
        
    def negative_sampling(self, embeddings, num_negatives=8):
        batch_size, num_sentences, num_args, embedding_dim = embeddings.size()
        all_negatives = []

        for i in range(batch_size):
            for j in range(num_sentences):
                # Flatten the arguments dimension to sample across all arguments in the sentence
                flattened_embeddings = embeddings[i, j].view(-1, embedding_dim)
                
                # Get indices of non-padded embeddings (assuming padding is represented by all-zero vectors)
                non_padded_indices = torch.where(torch.any(flattened_embeddings != 0, dim=1))[0]

                # Randomly sample negative indices from non-padded embeddings
                if len(non_padded_indices) > 0:
                    negative_indices = non_padded_indices[torch.randint(0, len(non_padded_indices), (num_negatives,))]
                else:
                    # If no non-padded embeddings, use zeros
                    negative_indices = torch.zeros(num_negatives, dtype=torch.long)

                negative_samples = flattened_embeddings[negative_indices, :]
                all_negatives.append(negative_samples)

        # Concatenate all negative samples into a single tensor
        all_negatives = torch.cat(all_negatives, dim=0)

        # If more samples than required, randomly select 'num_negatives' samples
        if all_negatives.size(0) > num_negatives:
            indices = torch.randperm(all_negatives.size(0))[:num_negatives]
            all_negatives = all_negatives[indices]

        return all_negatives
    
    def forward(self, sentence_ids, predicate_ids, arg0_ids, arg1_ids, tau):
        # Convert input IDs to embeddings
        sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings = self.aggregation(sentence_ids, predicate_ids, arg0_ids, arg1_ids)
        
        # Handle multiple spans by averaging predictions
        unsupervised_losses = torch.zeros((sentence_embeddings.size(0),), device=sentence_embeddings.device)
        
        # Creating storage for aggregated d tensors
        d_p_list, d_a0_list, d_a1_list = [], [], []
        
        negatives_p = self.negative_sampling(predicate_embeddings)
        negatives_a0 = self.negative_sampling(arg0_embeddings)
        negatives_a1 = self.negative_sampling(arg1_embeddings)

        # Process each sentence 
        for sentence_idx in range(sentence_embeddings.size(1)):
            s_sentence_span = sentence_embeddings[:, sentence_idx, :]

            d_p_sentence_list = []
            d_a0_sentence_list = []
            d_a1_sentence_list = []

            # Process each span
            for span_idx in range(predicate_embeddings.size(2)):                
                v_p_span = predicate_embeddings[:, sentence_idx, span_idx, :]
                v_a0_span = arg0_embeddings[:, sentence_idx, span_idx, :]
                v_a1_span = arg1_embeddings[:, sentence_idx, span_idx, :]

                # Feed the embeddings to the unsupervised module
                unsupervised_results = self.unsupervised(v_p_span, v_a0_span, v_a1_span, s_sentence_span, negatives_p, negatives_a0, negatives_a1, tau)                
                unsupervised_losses += unsupervised_results["loss"]
                
                if torch.isnan(unsupervised_results["loss"]).any():
                    print("loss is nan")
                
                # Use the vhat (reconstructed embeddings) for supervised predictions
                d_p_sentence_list.append(unsupervised_results['p']['d'])
                d_a0_sentence_list.append(unsupervised_results['a0']['d'])
                d_a1_sentence_list.append(unsupervised_results['a1']['d'])        


            # Aggregating across all spans
            d_p_sentence = torch.stack(d_p_sentence_list, dim=1)
            d_a0_sentence = torch.stack(d_a0_sentence_list, dim=1)
            d_a1_sentence = torch.stack(d_a1_sentence_list, dim=1)

            d_p_list.append(d_p_sentence)
            d_a0_list.append(d_a0_sentence)
            d_a1_list.append(d_a1_sentence)

        # Aggregating across all spans
        d_p_aggregated = torch.stack(d_p_list, dim=1)
        d_a0_aggregated = torch.stack(d_a0_list, dim=1)
        d_a1_aggregated = torch.stack(d_a1_list, dim=1)
        
        # Supervised predictions
        span_pred, sentence_pred = self.supervised(d_p_aggregated, d_a0_aggregated, d_a1_aggregated, sentence_embeddings)
    
        # Identify valid (non-nan) losses
        valid_losses = ~torch.isnan(unsupervised_losses)

        # Sum only the valid losses
        #unsupervised_loss = unsupervised_losses[valid_losses].sum()
        
        # Take average by summing the valid losses and dividing by num sentences so that padded sentences are also taken in equation
        unsupervised_loss = unsupervised_losses[valid_losses].sum() / sentence_embeddings.shape[1]
        
        return unsupervised_loss, span_pred, sentence_pred


# Set the necessary parameters
batch_size = 2
embedding_dim = 768
K = 14  # Number of frames/descriptors
num_frames = 14  # Assuming the number of frames is equal to K for simplicity
D_h = 512  # Dimension of the hidden representation
lambda_orthogonality = 0.1
M = 8
t = 8
tau = 1.0

# Define some mock token IDs data parameters
max_sentences_per_article = 8
max_sentence_length = 10
num_sentences = max_sentences_per_article
max_args_per_sentence = 3

# Generating mock token IDs for predicate, ARG0, ARG1, and their corresponding sentences
# We assume a vocab size of 30522 (standard BERT vocab size) for simplicity.
vocab_size = 30522

sentence_ids = torch.randint(0, vocab_size, (batch_size, max_sentences_per_article, max_sentence_length))
predicate_ids = torch.randint(0, vocab_size, (batch_size, max_sentences_per_article, max_args_per_sentence, max_sentence_length))
arg0_ids = torch.randint(0, vocab_size, (batch_size, max_sentences_per_article, max_args_per_sentence, max_sentence_length))
arg1_ids = torch.randint(0, vocab_size, (batch_size, max_sentences_per_article, max_args_per_sentence, max_sentence_length))

sentence_embeddings = torch.randn(batch_size, max_sentences_per_article, embedding_dim)
predicate_embeddings = torch.randn(batch_size, max_sentences_per_article, max_args_per_sentence, embedding_dim)
arg0_embeddings = torch.randn(batch_size, max_sentences_per_article, max_args_per_sentence, embedding_dim)
arg1_embeddings = torch.randn(batch_size, max_sentences_per_article, max_args_per_sentence, embedding_dim)

# Initialize the FRISS model
friss_model = FRISS(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K=K, num_frames=num_frames)

# Forward pass the mock data
unsupervised_loss, span_pred, sentence_pred = friss_model(sentence_ids, predicate_ids, arg0_ids, arg1_ids, 1)
unsupervised_loss, span_pred.shape, sentence_pred.shape

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(tensor(3.7889e+08, grad_fn=<DivBackward0>),
 torch.Size([2, 8, 14]),
 torch.Size([2, 14]))

# Train Model

The F1-Score (micro-averaged) and Average Precision Score are chosen as primary metrics for evaluating the multi-label classification task due to the following reasons:

1. **F1-Score (Micro)**:
    - The micro-averaged F1-score computes global counts of true positives, false negatives, and false positives. 
    - It provides a balance between precision (the number of correct positive results divided by the number of all positive results) and recall (the number of correct positive results divided by the number of positive results that should have been returned).
    - Given the imbalance in the label distribution observed in the dataset, the micro-averaged F1-score is robust against this imbalance, making it a suitable metric for optimization.

2. **Average Precision Score**:
    - This metric summarizes the precision-recall curve, giving a single value that represents the average of precision values at different recall levels.
    - It's especially valuable when class imbalances exist, as it gives more weight to the positive class (the rarer class in an imbalanced dataset).

Using these metrics will ensure that the model is optimized for a balanced performance across all labels, even if some labels are rarer than others.

In [102]:
import numpy as np
from sklearn.metrics import f1_score, average_precision_score
from math import exp
import json 
import csv

from torch.optim.lr_scheduler import StepLR

def train(model, train_dataloader, test_dataloader, optimizer, loss_function, alpha=0.5, num_epochs=10, tau_min=1, tau_decay=0.95, device='cuda', save_path='../notebooks/', save=False):
    tau = 1
    
    metrics = {
        'f1_span_micro': [],
        'f1_sentence_micro': [],
        'f1_span_macro': [],
        'f1_sentence_macro': []
    }
    
    scheduler = StepLR(optimizer, step_size=2, gamma=0.1)
    
    iteration = 0
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        model.train()
        
        total_loss = 0
        supervised_total_loss = 0
        unsupervised_total_loss = 0
        
        batch_progress = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc="Batches", leave=False)
        for batch_idx, batch in batch_progress:            
            iteration = iteration + 1
            
            optimizer.zero_grad()

            sentence_ids = batch['sentence_ids'].to(device)
            predicate_ids = batch['predicate_ids'].to(device)
            arg0_ids = batch['arg0_ids'].to(device)
            arg1_ids = batch['arg1_ids'].to(device)
            labels = batch['labels'].to(device)

            unsupervised_loss, span_logits, sentence_logits = model(sentence_ids, predicate_ids, arg0_ids, arg1_ids, tau)
                    
            span_loss = 0
            sentence_loss = 0

            # loop over spans
            for i in range(span_logits.size(1)):
                span_loss += loss_function(span_logits[:, i, :], labels.float())
            
            # span_loss = loss_function(span_logits, labels.float())            
            sentence_loss = loss_function(sentence_logits, labels.float())
            
            supervised_loss = span_loss + sentence_loss
            
            combined_loss = alpha * supervised_loss + (1-alpha) * unsupervised_loss
            
            if torch.isnan(combined_loss):
                print(f"NaN loss detected at epoch {epoch+1}, batch {batch_idx+1}. Stopping...")
                return
        
            combined_loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # After the backward pass
            if any(p.grad is not None and torch.isnan(p.grad).any() for p in model.parameters()):
                print(f"NaN gradients detected at epoch {epoch+1}, batch {batch_idx+1}. Stopping...")
                return
            
            optimizer.step()

            total_loss += combined_loss.item()
            supervised_total_loss += supervised_loss.item()
            unsupervised_total_loss += unsupervised_loss.item()

            batch_progress.set_description(f"Epoch {epoch+1} ({iteration}) Total Loss: {combined_loss.item():.3f}, SRLs: {span_loss:.3f}, Sentence: {sentence_loss:.3f}, CombinedS: {supervised_loss.item():.3f}, Unsupervised: {unsupervised_loss.item():.3f}")
                        
            if save:
                # Log metrics to CSV
                with open(save_path + 'training_metrics.csv', 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow([batch_idx, epoch+1, combined_loss.item(), supervised_loss.item(), unsupervised_loss.item()])

            # Explicitly delete tensors to free up memory
            del sentence_ids, predicate_ids, arg0_ids, arg1_ids, labels, unsupervised_loss
            torch.cuda.empty_cache()

        print(f"Epoch {epoch+1}/{num_epochs}, Combined Loss: {total_loss/len(train_dataloader)}, Supervised Loss: {supervised_total_loss/len(train_dataloader)}, Unsupervised Loss: {unsupervised_total_loss/len(train_dataloader)}")
        
        model.eval()
        
        span_preds = []
        sentence_preds = []
        combined_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in test_dataloader:
                sentence_ids = batch['sentence_ids'].to(device)
                predicate_ids = batch['predicate_ids'].to(device)
                arg0_ids = batch['arg0_ids'].to(device)
                arg1_ids = batch['arg1_ids'].to(device)
                labels = batch['labels'].to(device)
                
                _, span_logits, sentence_logits = model(sentence_ids, predicate_ids, arg0_ids, arg1_ids, tau)

                span_pred = []

                # loop over span and apply softmax
                for i in range(span_logits.size(1)):
                    span_pred.append((torch.softmax(span_logits[:, i, :], dim=1) > 0.5).float().cpu().numpy())

                sentence_pred = (torch.sigmoid(sentence_logits) > 0.5).float()
                
                span_preds.append(span_pred)
                sentence_preds.append(sentence_pred.cpu().numpy())
                combined_preds.append(combined_pred.cpu().numpy())
                all_labels.append(labels.cpu().numpy())

                # Explicitly delete tensors to free up memory
                del sentence_ids, predicate_ids, arg0_ids, arg1_ids, labels, span_logits, sentence_logits, span_pred, sentence_pred, combined_pred
                torch.cuda.empty_cache()

        all_span_preds = np.vstack(span_preds)
        all_sentence_preds = np.vstack(sentence_preds)
        all_combined_preds = np.vstack(combined_preds)
        all_labels = np.vstack(all_labels)

        f1_span_micro = f1_score(all_labels, all_span_preds, average='micro')
        f1_sentence_micro = f1_score(all_labels, all_sentence_preds, average='micro')
        
        f1_span_macro = f1_score(all_labels, all_span_preds, average='macro')
        f1_sentence_macro = f1_score(all_labels, all_sentence_preds, average='macro')

        metrics['f1_span_micro'].append(f1_span_micro)
        metrics['f1_sentence_micro'].append(f1_sentence_micro)
        
        metrics['f1_span_macro'].append(f1_span_macro)
        metrics['f1_sentence_macro'].append(f1_sentence_macro)

        print(f"Validation Metrics - micro F1 - Span/Sentence: {f1_span_micro:.4f}/{f1_sentence_micro:.4f}, macro F1 - Span/Sentence: {f1_span_macro:.4f}/{f1_sentence_macro:.4f}")
        
        # Anneal tau at the end of the epoch
        tau = max(tau_min, exp(-tau_decay * iteration))
        
        scheduler.step()
        
    if save:
        model_save_path = os.path.join(save_path, 'model1.pth')
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved to {model_save_path}")
    
        with open(os.path.join(save_path, 'metrics.json'), 'w') as f:
            json.dump(metrics, f)

    return metrics

# Dataset

In [103]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

num_sentences = 32
batch_size = 48
max_sentence_length = 64
max_arg_length = 12

train_dataset, test_dataset, train_dataloader, test_dataloader = get_datasets_dataloaders(df, tokenizer, recalculate_srl=False, batch_size=batch_size, max_sentences_per_article=num_sentences, max_sentence_length=max_sentence_length, max_arg_length=max_arg_length, pickle_path="../notebooks/FRISS_srl.pkl")

Load SRL from Pickle


  0%|          | 0/10211714 [00:00<?, ?it/s]

X: 6097
X_srl: 6097
y: 6097
CREATING DATASETS
TRAIN TEST SPLIT DONE
CREATION DONE


# Train

In [105]:
def get_friss_model(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob, bert_model_name="bert-base-uncased", load=True, path="", device='cuda'):
    """
    Loads the weights into an instance of the model class from the given path.
    
    Args:
    - model_class (torch.nn.Module): The class of the model (uninitialized).
    - path (str): Path to the saved weights.
    - device (str): Device to load the model on ('cpu' or 'cuda').
    
    Returns:
    - model (torch.nn.Module): Model with weights loaded.
    """

    # Model instantiation
    model = FRISS(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob=dropout_prob, bert_model_name=bert_model_name)
    model = model.to(device)
    
    if load:
        assert path != ""
        model.load_state_dict(torch.load(path, map_location=device))
    
    #model.eval()
    return model

In [106]:
torch.set_printoptions(profile="full")

import torch.optim as optim
import json
import csv

from tqdm.notebook import tqdm

# Hyperparameters
embedding_dim = 768
num_frames = 15

D_h = 768
lambda_orthogonality = 1e-3

K = num_frames
t = 8
M = 8
tau_min = 0.5
tau_decay = 5e-4

dropout_prob = 0.3

friss_model_path = "bert-base-uncased"
bert_model_path = "bert-base-uncased"

# Model instantiation
model = get_friss_model(embedding_dim, 
                        D_h, 
                        lambda_orthogonality, 
                        M, 
                        t, 
                        num_sentences, 
                        K, 
                        num_frames, 
                        dropout_prob=dropout_prob,
                        bert_model_name=bert_model_path,
                        load=False,
                        path=friss_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# LOSS

loss_function = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-4)#, weight_decay=1e-5)

# Train the model
alpha_value = 0.5
num_epochs_value = 10

save_path = "models/"

metrics = train(model, train_dataloader, test_dataloader, optimizer, loss_function, tau_min=tau_min, tau_decay=tau_decay, alpha=alpha_value, num_epochs=num_epochs_value, device=device, save=True, save_path=save_path)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

# Grid Search

In [38]:
from itertools import product
import torch.optim as optim
import csv

from tqdm.notebook import tqdm

# Hyperparameters
embedding_dim = 768

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def grid_search(train_dataloader, test_dataloader, search_space, num_epochs=10):
    # Store the results for each hyperparameter combination
    results = {}

    # Fixed values for K and num_frames
    K = 14
    num_frames = 14

    # Fixed values for dropout_prob and bert_model_name (adjust if necessary)
    bert_model_name = "../notebooks/models/fine-tuned-model/"

    # Initialize the file to write metrics
    with open("../notebooks/grid_search_metrics.csv", "w", newline='') as csvfile:
        fieldnames = ['combination', 'alpha', 'lr', 'D_h', 'lambda_orthogonality', 'M', 't', 'tau_min', 'tau_decay', 'dropout_prob', 'epoch', 'f1_span_micro', 'f1_span_macro', 'f1_sentence_micro', 'f1_sentence_macro', 'f1_combined_micro', 'f1_combined_macro']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Calculate the total number of combinations
        total_combinations = 1
        for key, values in search_space.items():
            total_combinations *= len(values)

        # Loop through all combinations
        for idx, combination in enumerate(product(*search_space.values())):
            print(f"Training combination {idx + 1}/{total_combinations}: {combination}")

            # Extract hyperparameters from the current combination
            alpha, lr, tau_min, tau_decay, t, D_h, lambda_orthogonality, M, dropout_prob = combination

            # Initialize the model with current hyperparameters
            model = FRISS(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob, bert_model_name)
            model.to(device)
        
                
            # Compute the `weight` parameter for each label
            label_frequencies = y.mean()
            weights = 1 / (label_frequencies + 1e-10)  # Adding a small value to avoid division by zero

            # Compute the `pos_weight` parameter
            pos_weights = (1 - label_frequencies) / (label_frequencies + 1e-10)

            # Convert the computed weights and pos_weights to PyTorch tensors
            weights_tensor = torch.tensor(weights.values, dtype=torch.float32).to(device)
            pos_weights_tensor = torch.tensor(pos_weights.values, dtype=torch.float32).to(device)

            loss_function = nn.BCEWithLogitsLoss(weight=weights_tensor, pos_weight=pos_weights_tensor, reduction="mean")
        
            # Define the optimizer
            optimizer = optim.AdamW(model.parameters(), lr=lr)

            # Define loss_function if needed (add this if your train function requires it)

            # Train the model with the current hyperparameters
            epoch_metrics = train(model, train_dataloader, test_dataloader, optimizer, loss_function, alpha=alpha, num_epochs=num_epochs, tau_min=tau_min, tau_decay=tau_decay, device=device, save=False)

            # Write the metrics to the CSV file
            for epoch in range(num_epochs):
                f1_span_micro = epoch_metrics['f1_span_micro'][epoch]
                f1_span_macro = epoch_metrics['f1_span_macro'][epoch]
                f1_sentence_micro = epoch_metrics['f1_sentence_micro'][epoch]
                f1_sentence_macro = epoch_metrics['f1_sentence_macro'][epoch]
                f1_combined_micro = epoch_metrics['f1_combined_micro'][epoch]
                f1_combined_macro = epoch_metrics['f1_combined_macro'][epoch]
                row = {
                    'combination': idx,
                    'alpha': alpha,
                    'lr': lr,
                    'D_h': D_h,
                    'lambda_orthogonality': lambda_orthogonality,
                    'M': M,
                    't': t,
                    'tau_min': tau_min,
                    'tau_decay': tau_decay,
                    'dropout_prob': dropout_prob,
                    'epoch': epoch + 1,
                    'f1_span_micro': f1_span_micro,
                    'f1_span_macro': f1_span_macro,
                    'f1_sentence_micro': f1_sentence_micro,
                    'f1_sentence_macro': f1_sentence_macro,
                    'f1_combined_micro': f1_combined_micro,
                    'f1_combined_macro': f1_combined_macro
                }
                writer.writerow(row)
                csvfile.flush()

    return results

search_space = {
    'alpha': [0.5, 0.2, 0.8],
    'lr': [1e-5, 2e-5, 5e-4, 1e-3],
    'tau_min': [0.5],
    'tau_decay': [5e-4],
    't': [5, 8, 10, 20],
    'D_h': [768, 768 * 2, 768 // 2, 768 * 3],
    'lambda_orthogonality': [1e-6, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2],
    'M': [5, 8, 10, 20],
    'dropout_rate': [0.1, 0.2, 0.3, 0.5]
}

# Call the grid search function
results = grid_search(train_dataloader, test_dataloader, search_space, 10)
results

Training combination 1/18432: (0.5, 1e-05, 0.5, 0.0005, 5, 768, 1e-06, 5, 0.1)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/10, Combined Loss: 10742.722412109375, Supervised Loss: 8.291224559148153, Unsupervised Loss: 21477.15380859375
Validation Metrics - micro F1 - Span/Sentence/Combined: 0.2967/0.3478/0.2987, macro F1 - Span/Sentence/Combined: 0.1632/0.2973/0.2817


...

Epoch 4/10, Combined Loss: 5055575.625, Supervised Loss: 7.190291881561279, Unsupervised Loss: 10111144.166666666
Validation Metrics - micro F1 - Span/Sentence/Combined: 0.4145/0.3230/0.3881, macro F1 - Span/Sentence/Combined: 0.2736/0.2711/0.3644


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 5/10, Combined Loss: 5050053.583333333, Supervised Loss: 6.922079046567281, Unsupervised Loss: 10100100.25
Validation Metrics - micro F1 - Span/Sentence/Combined: 0.4085/0.3178/0.3810, macro F1 - Span/Sentence/Combined: 0.2740/0.2643/0.3578


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Test model

In [42]:
def load_model_from_path(path, embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob, sentence_heads, srl_heads, device='cuda'):
    """
    Loads the weights into an instance of the model class from the given path.
    
    Args:
    - model_class (torch.nn.Module): The class of the model (uninitialized).
    - path (str): Path to the saved weights.
    - device (str): Device to load the model on ('cpu' or 'cuda').
    
    Returns:
    - model (torch.nn.Module): Model with weights loaded.
    """

    # Model instantiation
    model = FRISS(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob=dropout_prob, sentence_heads=sentence_heads, srl_heads=srl_heads)
    model = model.to(device)
    
    model.load_state_dict(torch.load(path, map_location=device))
    
    #model.eval()
    return model


In [43]:
# Hyperparameters
embedding_dim = 768
num_frames = 14

D_h = 768
lambda_orthogonality = 0.000001

K = 14
t = 5
M = 10
tau_min = 0.5
tau_decay = 5e-4

dropout_prob = 0.1

sentence_heads = 8
srl_heads = 7


model = load_model_from_path('models/model1.pth', embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob, sentence_heads, srl_heads)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
def predict(model, dataloader, y_columns, device='cuda'):
    """
    Make predictions with the given model and dataloader.
    
    Args:
    - model (torch.nn.Module): The model to make predictions with.
    - dataloader (DataLoader): DataLoader for the dataset to predict on.
    - y_columns (pandas.Index): Column names from the y dataframe which correspond to labels.
    - device (str): Device to make predictions on ('cpu' or 'cuda').
    
    Returns:
    - predicted_labels (list of lists): List containing the predicted labels for each instance.
    """
    model.eval()
    all_preds_span = []
    
    with torch.no_grad():
        for batch in dataloader:
            # Move data to device
            sentence_ids = batch['sentence_ids'].to(device)
            predicate_ids = batch['predicate_ids'].to(device)
            arg0_ids = batch['arg0_ids'].to(device)
            arg1_ids = batch['arg1_ids'].to(device)
            
            # Forward pass
            _, span_logits, sentence_logits, combined_logits = model(sentence_ids, predicate_ids, arg0_ids, arg1_ids, 0.6)
            combined_pred = (torch.sigmoid(combined_logits) > 0.5).float()

            all_preds_span.append(combined_pred.cpu().numpy())
                
            torch.cuda.empty_cache()

    predictions = np.vstack(all_preds_span)
    
    # Convert boolean predictions to labels
    predicted_labels = []
    for pred in predictions:
        labels = list(y_columns[pred.astype(bool)])
        predicted_labels.append(labels)
    
    return predicted_labels


In [46]:
import numpy as np

# article813452859
article = """Sadiq Khan Slammed for Pro-EU 'Message of Support' During Firework Display

The spectacular fireworks that lit up the London sky on Monday night caused a stir on social media over the display's pro-EU message, at a time when the nation is divided over its looming withdrawal from the bloc.
London Mayor Sadiq Khan faced mounting criticism after the capital's New Year's Eve fireworks display, which celebrated ties with the European Union, left a bad taste in the mouths of some Brits.
The 135-metre-high London Eye was lit up in blue while its tubs turned yellow, with the giant Ferris wheel resembling the star-studded flag of the European Union.Sadiq Khan called his fireworks display a "message of support" to EU citizens living in London.
"Our one million EU citizens are Londoners, they make a huge contribution, and no matter the outcome of Brexit — they will always be welcome", he said.
To the one million EU citizens who have made our city your home: you are Londoners, you make a huge contribution and you are welcome here.
I'm proud that tonight we will welcome in the new year with a message of support to you.
#LondonNYE #LondonIsOpen https://t.co/XctrgfXXaM — Sadiq Khan (@SadiqKhan) 31 декабря 2018 г.
However, a host of Londoners rushed to Twitter to accuse their mayor of "politicising" the celebrations — with some are even calling for his resignation.
I cannot believe this event has been politicised.
This man has no shame.
Just resign.
— wayne campbell (@campbs177) 31 декабря 2018 г.
Thanks a lot Sadiq Khan you ruined the fireworks display by talking about Europe, need I remind you about Brexit.
You have started of the new year by talking about relationships with the European Union.
Well done.
We need Boris Johnson back.
— Mitchell T Cannon (@MitchellTCanno1) 1 января 2019 г.
Another shameless attempt at using party politics on what is supposed to be a happy occasion — droneguy (@shelbyguitars) 1 января 2019 г.
Politicising another innocent event that should be no different to anyone no matter who they are or where they are from!
Shameful!
!
— Mike Dyer (@Miked2372Mike) 31 декабря 2018 г.
Someone was stabbed down the road from me last night.
How about sorting that stuff out instead of politicizing something that should be fun for everyone?How many times does it have to be said.
Commenting on Brexit isn't your job.
— Peter Rockett (@rockettp) 31 декабря 2018 г.
The UK voted to leave the EU in June 2016 via a nationwide referendum, with 51.9 per cent voting in favour of pulling out of the bloc, while 48.1 per cent wanted to remain.
The withdrawal is scheduled for the end of March; the Article 50 deadline.
The Remain sentiment dominated London, with nearly 60 percent of voters wanting Britain to stay in the European Union.
Sadiq Khan, an outspoken Remainer himself, earlier called for a second referendum on Brexit.
"The government's abject failure — and the huge risk we face of a bad deal or a 'no deal' Brexit — means that giving people a fresh say is now the right — and only — approach left for our country," he said in September.
"""

test_article = get_article_dataloader(article, tokenizer)
predict(model, test_article, y.columns)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Processed article 1/1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['Morality',
  'Security_and_defense',
  'Legality_Constitutionality_and_jurisprudence',
  'Political',
  'External_regulation_and_reputation']]

# Run test for validation

In [47]:
test_dataloader = get_test_dataloader(df_test["content"], tokenizer, batch_size=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Processed article 1/53
Processed article 2/53
Processed article 3/53
Processed article 4/53
Processed article 5/53
Processed article 6/53
Processed article 7/53
Processed article 8/53
Processed article 9/53
Processed article 10/53
Processed article 11/53
Processed article 12/53
Processed article 13/53
Processed article 14/53
Processed article 15/53
Processed article 16/53
Processed article 17/53
Processed article 18/53
Processed article 19/53
Processed article 20/53
Processed article 21/53
Processed article 22/53
Processed article 23/53
Processed article 24/53
Processed article 25/53
Processed article 26/53
Processed article 27/53
Processed article 28/53
Processed article 29/53
Processed article 30/53
Processed article 31/53
Processed article 32/53
Processed article 33/53
Processed article 34/53
Processed article 35/53
Processed article 36/53
Processed article 37/53
Processed article 38/53
Processed article 39/53
Processed article 40/53
Processed article 41/53
Processed article 42/53
P

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
predictions = predict(model, test_dataloader, y.columns)

In [49]:
df_preds = pd.DataFrame(predictions)

In [50]:
df_preds = pd.concat([df_test, df_preds], axis=1)

In [52]:
df_preds["pred_frames"] = df_preds.apply(lambda l: list([l[0], l[1], l[2], l[3], l[4], l[5]]), axis=1)

df_preds["pred_frames"] = df_preds["pred_frames"].apply(lambda l: ",".join([ f for f in l if f is not None]))

In [53]:
df_preds.to_csv("../notebooks/test.csv", sep="\t", index=False, columns=["article_id", "pred_frames"])

# Inspect dict

In [54]:
def inspect(model, dataloader, y_columns, device='cuda'):
    """
    Make predictions with the given model and dataloader.
    
    Args:
    - model (torch.nn.Module): The model to make predictions with.
    - dataloader (DataLoader): DataLoader for the dataset to predict on.
    - y_columns (pandas.Index): Column names from the y dataframe which correspond to labels.
    - device (str): Device to make predictions on ('cpu' or 'cuda').
    
    Returns:
    - predicted_labels (list of lists): List containing the predicted labels for each instance.
    """
    model.eval()
    
    all_preds_span = []
    
    # Initialize usage lists for each label
    num_labels = len(y_columns)
    all_used_labels_p = []
    all_used_labels_a0 = []
    all_used_labels_a1 = []
    
    with torch.no_grad():
        for batch in dataloader:
            used_labels_p = []
            used_labels_a0 = []
            used_labels_a1 = []
    
            # Move data to device
            sentence_ids = batch['sentence_ids'].to(device)
            predicate_ids = batch['predicate_ids'].to(device)
            arg0_ids = batch['arg0_ids'].to(device)
            arg1_ids = batch['arg1_ids'].to(device)
            
            sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings = model.aggregation(sentence_ids, predicate_ids, arg0_ids, arg1_ids)
            
            # Process each span
            for span_idx in range(sentence_embeddings.size(1)):
                s_sentence_span = sentence_embeddings[:, span_idx, :]
                v_p_span = predicate_embeddings[:, span_idx, :]
                v_a0_span = arg0_embeddings[:, span_idx, :]
                v_a1_span = arg1_embeddings[:, span_idx, :]
            
                #unsupervised.combined_autoencoder v_p, v_a0, v_a1, v_sentence, tau
                output = model.unsupervised.combined_autoencoder(v_p_span, v_a0_span, v_a1_span, s_sentence_span, 0.6)
                
                #print(output["p"]["g"].cpu().numpy())
                used_labels_p.append(output["p"]["g"].cpu().numpy())
                used_labels_a0.append(output["a0"]["g"].cpu().numpy())
                used_labels_a1.append(output["a1"]["g"].cpu().numpy())

            
            # Forward pass
            _, span_logits, sentence_logits, combined_logits = model(sentence_ids, predicate_ids, arg0_ids, arg1_ids, 0.6)
            combined_pred = (torch.sigmoid(combined_logits) > 0.5).float()

            all_preds_span.append(combined_pred.cpu().numpy())
                
            torch.cuda.empty_cache()
            
            all_used_labels_p.append(used_labels_p)
            all_used_labels_a0.append(used_labels_a0)
            all_used_labels_a1.append(used_labels_a1)

    predictions = np.vstack(all_preds_span)
    
    # Convert boolean predictions to labels
    predicted_labels = []
    for pred in predictions:
        labels = list(y_columns[pred.astype(bool)])
        predicted_labels.append(labels)
    
    return predicted_labels, all_used_labels_p, all_used_labels_a0, all_used_labels_a1

In [64]:
num_sentences = 32
batch_size = 1

train_dataset, test_dataset, train_dataloader, test_dataloader = get_datasets_dataloaders(X, y, tokenizer, recalculate_srl=False, batch_size=batch_size, max_sentences_per_article=num_sentences, max_sentence_length=64, max_arg_length=12, pickle_path="notebooks/X_srl_full.pkl")

Load SRL from Pickle
                                           Class  Train Distribution (%)  \
0                                       Morality               47.938144   
1                           Security_and_defense               42.783505   
2             Policy_prescription_and_evaluation               14.690722   
3   Legality_Constitutionality_and_jurisprudence               46.391753   
4                                       Economic                6.185567   
5                                      Political               54.381443   
6                           Crime_and_punishment               53.350515   
7             External_regulation_and_reputation               26.804124   
8                                 Public_opinion                5.670103   
9                          Fairness_and_equality               27.577320   
10                        Capacity_and_resources                6.443299   
11                               Quality_of_life               20.3

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [65]:
import numpy as np

predicted_labels, used_labels_p, used_labels_a0, used_labels_a1 = inspect(model, train_dataloader, y.columns)

In [66]:
len(used_labels_p)

388

In [73]:
categories = list(y.columns)

category_lists_p = {category: [] for category in categories}
category_lists_a1 = {category: [] for category in categories}
category_lists_a0 = {category: [] for category in categories}

loader = test_dataloader

for batch_idx in range(len(loader.dataset)):
    # Iterate over each sentence
    ds = loader.dataset[batch_idx]

    for sentence_idx in range(len(used_labels_p[batch_idx])):

        # Update the lists for each category
        for cat_idx, category in enumerate(categories):
            
            if used_labels_p[batch_idx][cat_idx][0][cat_idx] > 0.8:
                category_lists_p[category].append(ds["predicate_ids"][sentence_idx].numpy())
            
            if used_labels_a0[batch_idx][cat_idx][0][cat_idx] > 0.8:
                category_lists_a0[category].append(ds["arg0_ids"][sentence_idx].numpy())
                
            if used_labels_a1[batch_idx][cat_idx][0][cat_idx] > 0.8:
                category_lists_a1[category].append(ds["arg1_ids"][sentence_idx].numpy())

In [74]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

def decode_tokens(token_dict, stop_words):
    decoded_data = {}
    for category, token_lists in token_dict.items():
        decoded_data[category] = []
        for tokens in token_lists:
            if np.any(tokens > 0):
                # Decode the tokens
                decoded_text = tokenizer.decode(tokens, skip_special_tokens=True).strip()
                # Tokenize and remove stop words
                words = word_tokenize(decoded_text)
                filtered_words = [word for word in words if word.lower() not in stop_words]
                # Join the words back into a string
                decoded_data[category].append(' '.join(filtered_words))
    return decoded_data

stop_words = set(stopwords.words('english'))  # Assuming your text is in English

# Decode the token IDs for each ARG
decoded_predicate = decode_tokens(category_lists_p, stop_words)
decoded_arg0 = decode_tokens(category_lists_a0, stop_words)
decoded_arg1 = decode_tokens(category_lists_a1, stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
# Initialize a list to collect DataFrame rows
rows = []

# Populate the list with rows
for frame in set(decoded_predicate) | set(decoded_arg0) | set(decoded_arg1):
    # Get the lists, joining multiple words with a comma
    pred_words = ', '.join([ s.strip() for s in list(set(decoded_predicate.get(frame, []))) if s is not None or l != ""])
    arg0_words = ', '.join(list(set(decoded_arg0.get(frame, []))))
    arg1_words = ', '.join(list(set(decoded_arg1.get(frame, []))))

    # Create a dictionary for the row
    row = {
        "Frame": frame,
        "Predicate": pred_words,
        "ARG0": arg0_words,
        "ARG1": arg1_words
    }
    
    # Append the row dictionary to the rows list
    rows.append(row)

# Convert the list of rows to a DataFrame
df_full_table = pd.DataFrame(rows)

# Display the DataFrame
df_full_table.style.hide_index()

/tmp/ipykernel_66/3321433862.py:26: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  df_full_table.style.hide_index()


Frame,Predicate,ARG0,ARG1
Cultural_identity,", grants, prohibit, keep, used, back, stop, reports, occur, asked, breaking, bite, safeguard, agree, sending, said, opt",", individual, author, biblical illiterates, creator, god, picture inside seal, john hancock , first signer declaration, american history found document, speaker -, person, preachers, completing poll, caesar, whosoever",
Public_opinion,,"us, american government employee stationed southern city, mike pompeo , us secretary state,","“ would raise serious questions whether department, matter “ handled properly ”, justice department release special counsel robert mueller ’"
Policy_prescription_and_evaluation,,", nominee, ramirez, russians , chinese , iranians ,, cia ’ front corporations narcotics business, ambellas intellihub, laura loomer , noxious far -, haig 's attorney marc victor, clinton regime, gop, ’, people, haig, [ senate majority leader ] mitch mcconnell chuck grass, federalist society, senate judiciary committee chairman chuck grassley ( r -, search warrant records, house democratic leader nancy pelosi ( ca ), paddock, trump, american public, two women, ford, waste - filled us military / security complex, one western media source, us democratic party, president eisenhower, intellihub, douglas haig , 55 - year - old, democratic senator chuck schumer ( ny ), high court, insane neoconservatives ,, sum, president trump, bonkers conspiracy site intellihub, cia cut -, corrupt filth, new yorker magazine, douglas haig ’ @ linkedin account, attorneys, washington insiders, presstitute media, ambellas, leaders democratic party , nancy pe, george soros ’ money, customer, democrats, haig 's linkedin profile, democratic minority leader us house representatives, media - savvy sleazeball lawyer, mesa man, left",", man ’ actions, incident, access freedom outpost updates free charge, hat, poll - story, employer, questions concerns, jimenez cut party, entire event, meal, brett kavanaugh, america, jimenez worked “ part - time door, teenager wearing # maga hat, different opinions president, site 's privacy policy terms, aggression, bar, ugly head, altercation, boy ’ hat, field, since terminated employee , actions, rumble, business"
Crime_and_punishment,", mean, resumed, attack, preferred, run, launched, understand, ate, overthrew, need, vaporized, include, suspended, accelerate, take, ', called, travel, gotten, act, boasts, opt, shows, spending, uses, using, resumes, avoided, spread, moved, grants, made, means, add, relishing, fired, cajole, signed, talking, agree, threaten, mastered, deployed, skimped",", us south korea, pakistan ‘, us power, washington, administration ’ amateur foreign policymakers, north korea test, north koreans , eccentric, us military, president trump, israel, north korea, nuclear explosions, shatter japan cripple south korea, north korea ’ kim jong - un, “",
Fairness_and_equality,,", intellihub, douglas haig , 55 - year - old, ambellas, bonkers conspiracy site intellihub, paddock, cia cut -, ambellas intellihub, customer, laura loomer , noxious far -, haig 's attorney marc victor, haig 's linkedin profile, ’, mesa man, people, haig, douglas haig ’ @ linkedin account, search warrant records",
Economic,", resumed, need, suspended, ', travel, gotten, act, opt, shows, spending, using, resumes, grants, means, signed, agree, made, deployed",", two, clinton, us, jim acosta, completing poll, trump, order, us forces, military, swisher, others, military force, hillary clinton, migrant caravan, president trump, caravan , made mostly male hondur, ruptly video, president donald trump, migrants, former secretary state hillary clinton, hillary, democratic party",", image, attempt coup president, 1 , 000 billion dollar budget military, pelosi ’ false accusation, multiple shooters, trump ’ comments, trump putin, new information, junk, us, countless hours, inside @ mandalaybay valet center, donald trump, many unanswered questions dr, license plate numbers given police, la